# MLOps class final project 

### Creating mlrun project

In [1]:
import mlrun
project = mlrun.get_or_create_project(name='mlops', user_project=True, context='./')

> 2023-01-14 19:40:54,300 [info] Created and saved project mlops-jovyan: {'from_template': None, 'overwrite': False, 'context': './', 'save': True}
> 2023-01-14 19:40:54,303 [info] created project mlops and saved in MLRun DB


### Setting up functions

In [2]:
import os

# Setting get data function
get_data = mlrun.code_to_function(name='gen_dataset', kind='job', image='mlrun/mlrun', handler='get_data', filename='src/get_data.py')

# Setting feature-selection
feature_selection = mlrun.import_function('hub://feature_selection')

# Setting outlier removal function
outlier_removal = mlrun.code_to_function(name='outlier_removal', kind='job', image='mlrun/mlrun', handler='run', filename='src/outlier_removal.py')

# Setting dalex function
dalex = mlrun.code_to_function(name='dalex', kind='job', handler='run_dalex',filename='src/dalex.py')

# Setting training function
train = mlrun.code_to_function(name='train', kind='job', handler='train',filename='src/auto_trainer.py')


In [3]:
project.set_function(get_data)
project.set_function(feature_selection)
project.set_function(outlier_removal)
project.set_function(dalex)
project.set_function(train)

### Creating the kubeflow-pipeline

In [4]:
%%writefile src/trainer_baseline.py
import mlrun
from kfp import dsl
import sklearn
from src.outlier_removal import *

@dsl.pipeline(
    name="Automatic Pipeline",
    description="Train & Evaluate"
)
def kfpipeline(dataset: str='housing',
               path: str='/home/jovyan/data/src/housing.csv',
               label_column:str='MEDV',
               k: int=5,
               min_votes: float=3,
               remove_outlier:bool= False):
    
    project = mlrun.get_current_project()
    
    # @@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@
    # @@@@@@@@@@@@@@@@@@@@@@@@@@@@ Getting the data @@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@
    # @@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@
    get_data_run = mlrun.run_function(name='get_data',
                                      function='gen-dataset',
                                      params={'dataset': dataset,
                                              'path': path},
                                      outputs=[dataset])
    
    # @@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@
    # @@@@@@@@@@@@@@@@@@@@@@@@@@@@@@ feature selection @@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@
    # @@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@
    feature_selection_run = mlrun.run_function("hub://feature_selection",
                                               params={'ignore_type_errors': True,
                                                       "stat_filters": ['f_classif'
                                                                        ,'f_regression'
                                                                        ,'r_regression'
                                                                        ,'mutual_info_regression'],
                                                       "model_filters": {'AdaBoostRegressor':'AdaBoostRegressor',
                                                                         'ExtraTreesRegressor':'ExtraTreesRegressor',
                                                                         'GradientBoostingRegressor':'GradientBoostingRegressor',
                                                                         'RandomForestRegressor':'RandomForestRegressor',
                                                                         'RandomTreesEmbedding':'RandomTreesEmbedding',
                                                       },
                                                       "label_column": label_column,
                                                       "k": k,
                                                       "min_votes": min_votes},
                                               inputs={'df_artifact': get_data_run.outputs[dataset]},
                                               outputs=['feature_scores', 'selected_features_count',
                                                        'selected_features'])
    
    # @@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@
    # @@@@@@@@@@@@@@@@@@@@@@@@@@@@@@ outlier detection @@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@
    # @@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@
    # Setting outlier removal params
    votes_thresholds = 3
    pyod_contamination=0.2 # + (0,0.5)
    z_score_threshold=3
    iqr_low=0.01
    iqr_high=0.99
    iqr_max_removal_percent_per_column=0.95
    remove_outliers_functions = [(remove_outliers_z_score, {'threshold': z_score_threshold}),
                        (remove_outliers_iqr, {'low_quantile': iqr_low, 'high_quantile':iqr_high, 'max_removal_percent_per_column':iqr_max_removal_percent_per_column}),
                        (remove_outliers_LOF, {'contamination': pyod_contamination}),
                        (remove_outliers_ABOD, {'contamination': pyod_contamination}),
                           (remove_outliers_HBOS, {'contamination': pyod_contamination})
                            ]
    
    outlier_removal_run = mlrun.run_function(name='outlier_removal',
                                            function='outlier-removal',
                                            inputs={'dataitem': feature_selection_run.outputs['selected_features']},
                                            params={'remove_outliers_functions': remove_outliers_functions, 
                                                    'remove_outlier': remove_outlier,
                                                    'votes_thresholds': votes_thresholds,
                                                    'label_column': label_column,
                                                    'random_state': 50},
                                            outputs=['outlier_removal', 'outlier_removal_test'])
    
    # @@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@
    # @@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@ dalex @@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@
    # @@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@
    
    dalex = project.run_function(name='dalex',
                                 function='dalex',
                                 params={'df_train': outlier_removal_run.outputs['outlier_removal'],
                                         'df_test': outlier_removal_run.outputs['outlier_removal_test'],
                                         'target': label_column},
                                 outputs=['train_data', 'test_data', 'dalex_output'])
    
    params = {"model_class": "xgboost.XGBRegressor",
              "label_columns": label_column,
              "model_name": dataset + '_dalex'}
    try:
        for key,val in json.loads(mlrun.get_dataitem(dalex.outputs['dalex_output']).get()).items():
            params['sample_weight'] = val
    except:
        print('weights are deisabled')
    # @@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@
    # @@@@@@@@@@@@@@@@@@@@@@@@@@@@@@ Training the model @@@@@@@@@@@@@@@@@@@@@@@@@@@@@@
    # @@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@    
    # Train a model using the auto_trainer hub function
    train_run = mlrun.run_function(name= 'train',
                                   function='train',
                                   inputs={"dataset": dalex.outputs['train_data'], 
                                           "test_set": dalex.outputs['test_data']},
                                   params = params, 
                                   handler='train',
                                   outputs=["model"],
                               )

Writing src/trainer_baseline.py


### Running the pipeline with mlrun project

In [5]:
# Register the workflow file:
workflow_name = "trainer_baseline"
project.set_workflow(workflow_name, "src/trainer_baseline.py")

# Save the project:
project.save()

In [6]:
run = project.run(name=workflow_name,watch=False,local=True, overwrite=True,
                  arguments={'remove_outlier': True,
                             'path': '/home/jovyan/data/MLOps22/project/src/housing.csv',
                             'k': 7,
                             "min_votes":3})

> 2023-01-14 19:40:57,003 [warning] it is recommended to use k8s secret (specify secret_name), specifying the aws_access_key/aws_secret_key directly is unsafe
> 2023-01-14 19:40:57,008 [warning] it is recommended to use k8s secret (specify secret_name), specifying the aws_access_key/aws_secret_key directly is unsafe
> 2023-01-14 19:40:57,014 [warning] it is recommended to use k8s secret (specify secret_name), specifying the aws_access_key/aws_secret_key directly is unsafe
> 2023-01-14 19:40:57,017 [warning] it is recommended to use k8s secret (specify secret_name), specifying the aws_access_key/aws_secret_key directly is unsafe
> 2023-01-14 19:40:57,021 [warning] it is recommended to use k8s secret (specify secret_name), specifying the aws_access_key/aws_secret_key directly is unsafe


> 2023-01-14 19:40:57,032 [info] starting run get_data uid=06a1ca77ba94460fb8e3b4b4a31d5510 DB=http://mlrun-api:8080


project,uid,iter,start,state,name,labels,inputs,parameters,results,artifacts
mlops-jovyan,...a31d5510,0,Jan 14 19:40:57,completed,get_data,workflow=198e3444dcfe4b0488659347193a06eakind=owner=jovyanhost=mlrun-jupyter-5cd9c659c-2dpxf,,dataset=housingpath=/home/jovyan/data/MLOps22/project/src/housing.csv,,housing


> 2023-01-14 19:40:57,852 [info] run executed, status=completed
> 2023-01-14 19:40:58,266 [warning] it is recommended to use k8s secret (specify secret_name), specifying the aws_access_key/aws_secret_key directly is unsafe
> 2023-01-14 19:40:58,273 [info] starting run feature-selection-feature_selection uid=8c99a4f969a843439b6c8f0dd86ae1e7 DB=http://mlrun-api:8080
> 2023-01-14 19:41:05,422 [info] votes needed to be selected: 3


project,uid,iter,start,state,name,labels,inputs,parameters,results,artifacts
mlops-jovyan,...d86ae1e7,0,Jan 14 19:40:58,completed,feature-selection-feature_selection,workflow=198e3444dcfe4b0488659347193a06eakind=owner=jovyanhost=mlrun-jupyter-5cd9c659c-2dpxf,df_artifact,"ignore_type_errors=Truestat_filters=['f_classif', 'f_regression', 'r_regression', 'mutual_info_regression']model_filters={'AdaBoostRegressor': 'AdaBoostRegressor', 'ExtraTreesRegressor': 'ExtraTreesRegressor', 'GradientBoostingRegressor': 'GradientBoostingRegressor', 'RandomForestRegressor': 'RandomForestRegressor', 'RandomTreesEmbedding': 'RandomTreesEmbedding'}label_column=MEDVk=7min_votes=3",,f_classiff_regressionr_regressionmutual_info_regressionAdaBoostRegressorExtraTreesRegressorGradientBoostingRegressorRandomForestRegressorRandomTreesEmbeddingfeature_scoresmax_scaled_scores_feature_scoresselected_features_countselected_features


...

> 2023-01-14 19:41:21,647 [info] run executed, status=completed
weights are deisabled
> 2023-01-14 19:41:21,651 [info] starting run train uid=2b049420005c48038b55afc4be6b20b1 DB=http://mlrun-api:8080
> 2023-01-14 19:41:21,837 [info] Sample set not given, using the whole training set as the sample set
> 2023-01-14 19:41:21,891 [info] training 'housing_dalex'


project,uid,iter,start,state,name,labels,inputs,parameters,results,artifacts
mlops-jovyan,...be6b20b1,0,Jan 14 19:41:21,completed,train,workflow=198e3444dcfe4b0488659347193a06eakind=owner=jovyanhost=mlrun-jupyter-5cd9c659c-2dpxf,datasettest_set,model_class=xgboost.XGBRegressorlabel_columns=MEDVmodel_name=housing_dalex,mean_absolute_error=2.2533614177329864r2_score=0.8462224970882577root_mean_squared_error=3.4387234454168936mean_squared_error=11.82481893405983,feature-importancetest_setmodel


> 2023-01-14 19:41:22,542 [info] run executed, status=completed


uid,start,state,name,parameters,results
...a31d5510,Jan 14 19:40:57,completed,get_data,dataset=housingpath=/home/jovyan/data/MLOps22/project/src/housing.csv,
...d86ae1e7,Jan 14 19:40:58,completed,feature-selection-feature_selection,"ignore_type_errors=Truestat_filters=['f_classif', 'f_regression', 'r_regression', 'mutual_info_regression']model_filters={'AdaBoostRegressor': 'AdaBoostRegressor', 'ExtraTreesRegressor': 'ExtraTreesRegressor', 'GradientBoostingRegressor': 'GradientBoostingRegressor', 'RandomForestRegressor': 'RandomForestRegressor', 'RandomTreesEmbedding': 'RandomTreesEmbedding'}label_column=MEDVk=7min_votes=3",
...a3839358,Jan 14 19:41:05,completed,outlier_removal,"remove_outliers_functions=[(, {'threshold': 3}), (, {'low_quantile': 0.01, 'high_quantile': 0.99, 'max_removal_percent_per_column': 0.95}), (, {'contamination': 0.2}), (, {'contamination': 0.2}), (, {'contamination': 0.2})]remove_outlier=Truevotes_thresholds=3label_column=MEDVrandom_state=50",
...ad466e1f,Jan 14 19:41:08,completed,dalex,df_train=store://artifacts/mlops-jovyan/outlier_removal_outlier_removal:198e3444dcfe4b0488659347193a06eadf_test=store://artifacts/mlops-jovyan/outlier_removal_outlier_removal_test:198e3444dcfe4b0488659347193a06eatarget=MEDV,
...be6b20b1,Jan 14 19:41:21,completed,train,model_class=xgboost.XGBRegressorlabel_columns=MEDVmodel_name=housing_dalex,mean_absolute_error=2.2533614177329864r2_score=0.8462224970882577root_mean_squared_error=3.4387234454168936mean_squared_error=11.82481893405983


> 2023-01-14 19:41:22,570 [info] started run workflow mlops-jovyan-trainer_baseline with run id = '198e3444dcfe4b0488659347193a06ea' by local engine


In [7]:
run = project.run(name=workflow_name,watch=False,local=True, overwrite=True,
                  arguments={'dataset': 'motor',
                             'path': '/home/jovyan/data/MLOps22/project/src/freMTPL2freq.csv',
                             'label_column': 'ClaimNb',
                             'remove_outlier': True,
                             'k': 7, 
                             "min_votes":3})

> 2023-01-14 19:41:22,643 [info] starting run get_data uid=adb6bb4793b74f829dc6ae47e1d0ec39 DB=http://mlrun-api:8080


project,uid,iter,start,state,name,labels,inputs,parameters,results,artifacts
mlops-jovyan,...e1d0ec39,0,Jan 14 19:41:22,completed,get_data,workflow=5f347ecb895143f681121e86d9701b69kind=owner=jovyanhost=mlrun-jupyter-5cd9c659c-2dpxf,,dataset=motorpath=/home/jovyan/data/MLOps22/project/src/freMTPL2freq.csv,,motor


> 2023-01-14 19:41:28,798 [info] run executed, status=completed
> 2023-01-14 19:41:29,089 [warning] it is recommended to use k8s secret (specify secret_name), specifying the aws_access_key/aws_secret_key directly is unsafe
> 2023-01-14 19:41:29,096 [info] starting run feature-selection-feature_selection uid=aa46f1b7434044839c9685815291a991 DB=http://mlrun-api:8080
> 2023-01-14 19:55:34,665 [info] votes needed to be selected: 3


project,uid,iter,start,state,name,labels,inputs,parameters,results,artifacts
mlops-jovyan,...5291a991,0,Jan 14 19:41:29,completed,feature-selection-feature_selection,workflow=5f347ecb895143f681121e86d9701b69kind=owner=jovyanhost=mlrun-jupyter-5cd9c659c-2dpxf,df_artifact,"ignore_type_errors=Truestat_filters=['f_classif', 'f_regression', 'r_regression', 'mutual_info_regression']model_filters={'AdaBoostRegressor': 'AdaBoostRegressor', 'ExtraTreesRegressor': 'ExtraTreesRegressor', 'GradientBoostingRegressor': 'GradientBoostingRegressor', 'RandomForestRegressor': 'RandomForestRegressor', 'RandomTreesEmbedding': 'RandomTreesEmbedding'}label_column=ClaimNbk=7min_votes=3",,f_classiff_regressionr_regressionmutual_info_regressionAdaBoostRegressorExtraTreesRegressorGradientBoostingRegressorRandomForestRegressorRandomTreesEmbeddingfeature_scoresmax_scaled_scores_feature_scoresselected_features_countselected_features


> 2023-01-14 19:55:35,129 [info] run executed, status=completed
> 2023-01-14 19:55:35,134 [info] starting run outlier_removal uid=304510ce0502441b98a40c512fb4c02b DB=http://mlrun-api:8080
Removed: 1985
> 2023-01-14 19:58:16,427 [info] Outlier removal function removed successfully 1654


...

> 2023-01-14 20:03:14,276 [info] run executed, status=completed
weights are deisabled
> 2023-01-14 20:03:14,280 [info] starting run train uid=e333540252e44d268f427e1cf8048cf5 DB=http://mlrun-api:8080
> 2023-01-14 20:03:14,540 [info] Sample set not given, using the whole training set as the sample set
> 2023-01-14 20:03:14,609 [info] training 'motor_dalex'


project,uid,iter,start,state,name,labels,inputs,parameters,results,artifacts
mlops-jovyan,...f8048cf5,0,Jan 14 20:03:14,completed,train,workflow=5f347ecb895143f681121e86d9701b69kind=owner=jovyanhost=mlrun-jupyter-5cd9c659c-2dpxf,datasettest_set,model_class=xgboost.XGBRegressorlabel_columns=ClaimNbmodel_name=motor_dalex,mean_absolute_error=0.09739165967394417r2_score=0.034121066137656086root_mean_squared_error=0.23298860328114113mean_squared_error=0.05428368925889697,feature-importancetest_setmodel


> 2023-01-14 20:03:37,977 [info] run executed, status=completed


uid,start,state,name,parameters,results
...e1d0ec39,Jan 14 19:41:22,completed,get_data,dataset=motorpath=/home/jovyan/data/MLOps22/project/src/freMTPL2freq.csv,
...5291a991,Jan 14 19:41:29,completed,feature-selection-feature_selection,"ignore_type_errors=Truestat_filters=['f_classif', 'f_regression', 'r_regression', 'mutual_info_regression']model_filters={'AdaBoostRegressor': 'AdaBoostRegressor', 'ExtraTreesRegressor': 'ExtraTreesRegressor', 'GradientBoostingRegressor': 'GradientBoostingRegressor', 'RandomForestRegressor': 'RandomForestRegressor', 'RandomTreesEmbedding': 'RandomTreesEmbedding'}label_column=ClaimNbk=7min_votes=3",
...2fb4c02b,Jan 14 19:55:35,completed,outlier_removal,"remove_outliers_functions=[(, {'threshold': 3}), (, {'low_quantile': 0.01, 'high_quantile': 0.99, 'max_removal_percent_per_column': 0.95}), (, {'contamination': 0.2}), (, {'contamination': 0.2}), (, {'contamination': 0.2})]remove_outlier=Truevotes_thresholds=3label_column=ClaimNbrandom_state=50",
...4e5ab682,Jan 14 19:58:17,completed,dalex,df_train=store://artifacts/mlops-jovyan/outlier_removal_outlier_removal:5f347ecb895143f681121e86d9701b69df_test=store://artifacts/mlops-jovyan/outlier_removal_outlier_removal_test:5f347ecb895143f681121e86d9701b69target=ClaimNb,
...f8048cf5,Jan 14 20:03:14,completed,train,model_class=xgboost.XGBRegressorlabel_columns=ClaimNbmodel_name=motor_dalex,mean_absolute_error=0.09739165967394417r2_score=0.034121066137656086root_mean_squared_error=0.23298860328114113mean_squared_error=0.05428368925889697


> 2023-01-14 20:03:38,008 [info] started run workflow mlops-jovyan-trainer_baseline with run id = '5f347ecb895143f681121e86d9701b69' by local engine
